# Case Study 6 - National wheat yield and efficiency
## Description 
As a development corporation, I want to combine annual wheat tonnes (data at the SA2 level) with monthly rainfall data (data at the SA2 level), so I can calculate national wheat water use efficiency. 
## Case Breakdown 
- **Actors:** Development corporation
- **Goals:** Based on the historical data for predictions
- **Scope** National
## Generalised case
I want to combine a suite of spatial variables at different scales across multiple sites so I can analyse the factors correlated with a variable of interest. (Subcase of Case Study 1)
## Comparable cases
- I want to aggregate iMapPests data for the same pest across multiple sites and locations so I can analyse the relationship between population levels and environmental context at the time and over the previous month, including weather (temperature, rainfall, humidity - all xyt), lunar phase (t) and greenness (xyt - see https://portal.tern.org.au/metadata/TERN/8542d90e-6e20-4ad8-b30d-0a171b61d3f5).
## Stakeholders 
- **Name:** Jeff Cumpston
- **Contact:** jeff.cumpston@grdc.com.au


## Data Sources
To be confirmed

- BOM monthly rainfall grid data: http://www.bom.gov.au/climate/averages/climatology/average-rainfall-metadata.shtml - may need to request access

## Data Transformation
1. Import SA2 shape files and associated yield data for a given season.
2. Import monthly rainfall for all months in growing season (which?)

In [4]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE

## DataCube Generation
3. Derive total seasonal rainfall per SA2 area
4. Normalise yield and rainfall to 1km2 means

In [5]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE

## Data Analysis/Visualisation
5. Report yield / rainfall for all relevant SA2 areas 

In [6]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE